# Lab 6.1: AggDemo

## Tổng quan bài tập
**Đề bài**: Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1Xi_kx_iyE96qTmLU8kb4L4GtRAx0BFF5/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file .csv có cấu trúc như sau:
```
root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
```

# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [3]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# AggDemo

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f


conf = SparkConf() \
    .setMaster('local') \
    .setAppName('AggDemo')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = "/content/gdrive/My Drive/DEP303/invoices.csv"

Đọc dữ liệu từ Dataset

In [5]:
invoice_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(DATASET_PATH)

invoice_df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     NULL|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

Hãy sử dụng các thao tác Aggregration để tạo một Dataframe mới với cấu trúc như sau:
- `Count`: Số lượng các dòng trong dataset.
- `TotalQuantity`: Tổng của tất cả các trường `Quantity` trong dataset.
- `AvgPrice`: Trung bình cộng của tất cả các trường `UnitPrice` trong dataset.
- `CountDistinct`: Số lượng giá trị riêng biệt của trường `InvoiceNo`.

In [6]:
invoice_df.select(f.count("*").alias("Count *"),
                  f.sum("Quantity").alias("TotalQuantity"),
                  f.avg("UnitPrice").alias("AvgPrice"),
                  f.countDistinct("InvoiceNo").alias("CountDistinct")).show()

+-------+-------------+------------------+-------------+
|Count *|TotalQuantity|          AvgPrice|CountDistinct|
+-------+-------------+------------------+-------------+
| 541909|      5176450|4.6111136260884855|        25900|
+-------+-------------+------------------+-------------+



Hãy sử dụng các thao tác Aggregration để tạo một Dataframe mới với cấu trúc như sau:
- `Count`: Số lượng các dòng trong dataset.
- `TotalQuantity`: Tổng của tất cả các trường `Quantity` trong dataset.
- `AvgPrice`: Trung bình cộng của tất cả các trường `UnitPrice` trong dataset.

Ở phần này, bạn hãy sử dụng dưới dạng String Expression

In [7]:
invoice_df.selectExpr("count(1) as Count",
                      "sum(Quantity) as TotalQuantity",
                      "avg(UnitPrice) as AvgPrice").show()

+------+-------------+-----------------+
| Count|TotalQuantity|         AvgPrice|
+------+-------------+-----------------+
|541909|      5176450|4.611113626083471|
+------+-------------+-----------------+



Hãy sử dụng các thao tác Aggregration để tạo một Dataframe mới với cấu trúc như sau, dữ liệu trong Dataframe này sẽ được gộp nhóm theo hai trường `Country` và `InvoiceNo`:
- `Country`: Tên đất nước.
- `InvoiceNo`: Mã số hóa đơn.
- `TotalQuantity`: Tổng của tất cả các trường `Quantity`.
- `InvoiceValue`: Tổng giá trị của hóa đơn, mỗi hóa đơn sẽ có `Value = Quantity * UnitPrice`. Hãy làm tròn giá trị này thành 2 chữ số thập phân.

Ở phần này, bạn hãy sử dụng dưới dạng SQL

In [8]:
invoice_df.createOrReplaceTempView("invoices")
summary_sql = spark.sql("""SELECT Country,
                                  InvoiceNo,
                                  SUM(Quantity) AS TotalQuantity,
                                  ROUND(SUM(Quantity*UnitPrice),2) AS InvoiceValue
                           FROM invoices
                           GROUP BY Country, InvoiceNo""")

summary_sql.show()

+--------------+---------+-------------+------------+
|       Country|InvoiceNo|TotalQuantity|InvoiceValue|
+--------------+---------+-------------+------------+
|United Kingdom|   536446|          329|      440.89|
|United Kingdom|   536508|          216|      155.52|
|United Kingdom|   537018|           -3|         0.0|
|United Kingdom|   537401|          -24|         0.0|
|United Kingdom|   537811|           74|      268.86|
|United Kingdom|  C537824|           -2|       -14.9|
|United Kingdom|   538895|          370|      247.38|
|United Kingdom|   540453|          341|      302.45|
|United Kingdom|   541291|          217|      305.81|
|United Kingdom|   542551|           -1|         0.0|
|United Kingdom|   542576|           -1|         0.0|
|United Kingdom|   542628|            9|      132.35|
|United Kingdom|   542886|          199|      320.51|
|United Kingdom|   542907|           75|      313.85|
|United Kingdom|   543131|          134|       164.1|
|United Kingdom|   543189|  

Tương tự với đề bài bên trên, nhưng bạn hãy sử dụng Object Expression để thực hiện.

In [9]:
summary_df = invoice_df \
    .groupBy("Country", "InvoiceNo") \
    .agg(f.sum("Quantity").alias("TotalQuantity"),
         f.round(f.sum(f.expr("Quantity*UnitPrice")), 2).alias("InvoiceValue"))

summary_df.show()

+--------------+---------+-------------+------------+
|       Country|InvoiceNo|TotalQuantity|InvoiceValue|
+--------------+---------+-------------+------------+
|United Kingdom|   536446|          329|      440.89|
|United Kingdom|   536508|          216|      155.52|
|United Kingdom|   537018|           -3|         0.0|
|United Kingdom|   537401|          -24|         0.0|
|United Kingdom|   537811|           74|      268.86|
|United Kingdom|  C537824|           -2|       -14.9|
|United Kingdom|   538895|          370|      247.38|
|United Kingdom|   540453|          341|      302.45|
|United Kingdom|   541291|          217|      305.81|
|United Kingdom|   542551|           -1|         0.0|
|United Kingdom|   542576|           -1|         0.0|
|United Kingdom|   542628|            9|      132.35|
|United Kingdom|   542886|          199|      320.51|
|United Kingdom|   542907|           75|      313.85|
|United Kingdom|   543131|          134|       164.1|
|United Kingdom|   543189|  